<span style="font-family: Open Sans; font-weight: 800; font-size: 40px; font-style: normal; text-align:center;">

Especialización Inteligencia Artificial - CEIA

</span>

<span style="font-family: Open Sans; font-weight: 700; font-size: 30px; font-style: normal; text-align:center;">

Visión por Computadora I

Trabajo práctico Nro. III

</span>

<span style="font-family: Open Sans; font-weight: 400; font-size: 20px; font-style: normal;">




</span>

# Tabla de Contenido

[Consigna](#Consigna)<br>
[Punto I](#Parte-I)<br>
[Punto II](#Parte-II)<br>
[Punto III](#Parte-II)<br>

# Consigna

▪ Encontrar el logotipo de la gaseosa dentro de las imágenes provistas en
Material_TPs/TP3/images a partir del template Material_TPs/TP3/template.

1. (4 puntos) Obtener una detección del logo en cada imagen sin falsos positivos
2. (4 puntos) Plantear y validar un algoritmo para múltiples detecciones en la imagen
coca_multi.png con el mismo témplate del ítem 1
3. (2 puntos) Generalizar el algoritmo del item 2 para todas las imágenes.
Visualizar los resultados con bounding boxes en cada imagen mostrando el nivel de confianza
de la detección.

[inicio](#Tabla-de-contenido)

# Punto 0

[inicio](#Tabla-de-contenido)

In [91]:
# Importamos librerías
%matplotlib inline  
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt

In [138]:
# Cargamos el Template en escala de grises
template = cv.imread('material_TP3/template/pattern.PNG', cv.IMREAD_GRAYSCALE)

# Cargamos las imágenes
img1 = cv.imread('material_TP3/images/coca_logo_1.PNG')
img2 = cv.imread('material_TP3/images/coca_logo_2.PNG')
img3 = cv.imread('material_TP3/images/COCA-COLA-LOGO.jpg')
img4 = cv.imread('material_TP3/images/coca_multi.png')
img5 = cv.imread('material_TP3/images/coca_retro_1.png')
img6 = cv.imread('material_TP3/images/coca_retro_2.png')
img7 = cv.imread('material_TP3/images/logo_1.png')


"""
# Las mostramos para chequear que se cargaron bien
cv.imshow('Imagen Original 1 - Template', template)

cv.imshow('Imagen Original 1 - coca_logo_1', img1)
cv.imshow('Imagen Original 2 - coca_logo_2', img2)
cv.imshow('Imagen Original 3 - coca_multi', img3)
cv.imshow('Imagen Original 4 - coca_retro_1', img4)
cv.imshow('Imagen Original 5 - coca_retro_2', img5)
cv.imshow('Imagen Original 6 - COCA-COLA-LOGO', img6)
cv.imshow('Imagen Original 7 - logo_1', img7)

cv.waitKey(0)
cv.destroyAllWindows()
"""

"\n# Las mostramos para chequear que se cargaron bien\ncv.imshow('Imagen Original 1 - Template', template)\n\ncv.imshow('Imagen Original 1 - coca_logo_1', img1)\ncv.imshow('Imagen Original 2 - coca_logo_2', img2)\ncv.imshow('Imagen Original 3 - coca_multi', img3)\ncv.imshow('Imagen Original 4 - coca_retro_1', img4)\ncv.imshow('Imagen Original 5 - coca_retro_2', img5)\ncv.imshow('Imagen Original 6 - COCA-COLA-LOGO', img6)\ncv.imshow('Imagen Original 7 - logo_1', img7)\n\ncv.waitKey(0)\ncv.destroyAllWindows()\n"

-----------------------------------------

# Punto 1

[inicio](#Tabla-de-contenido)

▪ Encontrar el logotipo de la gaseosa dentro de las imágenes provistas en
Material_TPs/TP3/images a partir del template Material_TPs/TP3/template.

1. (4 puntos) Obtener una detección del logo en cada imagen sin falsos positivos

In [159]:
# Funciones Punto 1

def detectar_bordes_y_suprimir(img, thresh1, thresh2):
    bordes = cv.Canny(img, thresh1, thresh2)
    return np.sum(bordes) > 0

def aplicar_filtro_canny_en_region(img_gray, x, y, w, h, threshold1, threshold2):
    roi = img_gray[y:y+h, x:x+w]
    bordes = cv.Canny(roi, threshold1 = threshold1, threshold2 = threshold2)
    return np.sum(bordes) > 0

def find_template_matches(img_gray, template_redimensionado, threshold):
    w, h = template_redimensionado.shape[::-1]
    res = cv.matchTemplate(img_gray, template_redimensionado, cv.TM_CCOEFF_NORMED)
    loc = np.where(res >= threshold)
    return loc, res


def detect_and_visualize(img ,template, template_w, template_h, threshold, inv_template):
    
    # Pasamos la imagen a escala de grises
    img_gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    
    # Invertimos colores del template si es necesario
    if inv_template:
        template = cv.bitwise_not(template)
    
    # Redimensionamos el template
    template_redimensionado = cv.resize(template, (template_w, template_h))
    loc, res = find_template_matches(img_gray, template_redimensionado, threshold) 

    w = template_w
    h = template_h
    
    # Aplicamos supresión de no máximos
    detecciones = []
    for pt in zip(*loc[::-1]):
        x, y = pt
        if not any(x - w <= xx <= x + w and y - h <= yy <= y + h for xx, yy in detecciones):
            if aplicar_filtro_canny_en_region(img_gray, x, y, w, h, threshold1=140, threshold2=150):
                detecciones.append((x, y))
    
    # Dibujamos el rectángulo en las detecciones después de la supresión de no máximos y el filtro de contorno
    for pt in detecciones:
        cv.rectangle(img, pt, (pt[0] + w, pt[1] + h), (0, 255, 0), 2)
    
    # Mostramos imagen con detecciones marcadas y nivel de confianza
    for pt in detecciones:
        conf = res[pt[1], pt[0]]
        cv.putText(img, f'Conf: {conf:.2f}', (pt[0], pt[1] - 10), cv.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
    
    cv.imshow("Detecciones", img)
    cv.waitKey(0)
    cv.destroyAllWindows()

In [161]:
# Punto 1

# Llamamos a la función

detect_and_visualize(img1, template, 135, 60, threshold=0.2, inv_template = True)
detect_and_visualize(img2, template, 170, 95, threshold=0.20, inv_template = True)
detect_and_visualize(img3, template, 1200, 400, threshold=0.195, inv_template = True)
detect_and_visualize(img4, template, 90, 40, threshold=0.47, inv_template = True)
detect_and_visualize(img5, template, 500, 165, threshold=0.29, inv_template = False)
detect_and_visualize(img6, template, 150, 50, threshold=0.37, inv_template = True)
detect_and_visualize(img7, template, 260, 100, threshold=0.237, inv_template = True)

-------------------------------------

# Punto 2

[inicio](#Tabla-de-contenido)

2. (4 puntos) Plantear y validar un algoritmo para múltiples detecciones en la imagen
coca_multi.png con el mismo témplate del ítem 1

In [29]:
# Funciones Punto 2

def detectar_bordes_y_suprimir(imagen, thresh1, thresh2):
    bordes = cv.Canny(imagen, thresh1, thresh2)
    return np.sum(bordes) > 0

def encontrar_mejor_coincidencia_template(img_gris, template_escalado, threshl):
    res = cv.matchTemplate(img_gris, template_escalado, cv.TM_CCOEFF_NORMED)
    loc = np.where(res >= threshl)
    
    detecciones = set()
    for pt in zip(*loc[::-1]):
        x, y = pt
        w, h = template_escalado.shape[::-1]
        
        if not any(x - w <= xx <= x + w and y - h <= yy <= y + h for xx, yy in detecciones):
            roi = img_gris[y:y+h, x:x+w]
            if detectar_bordes_y_suprimir(roi, thresh1=140, thresh2=150):
                detecciones.add((x, y))
    
    return detecciones

In [30]:
# Punto 2

def punto_2(ruta_imagen, tamaños_template, template, template_gris, template_inv):
    img_rgb = cv.imread(ruta_imagen)
    img_gris = cv.cvtColor(img_rgb, cv.COLOR_BGR2GRAY)

    mejor_medida = None
    max_deteccion = 0
    threshold = 0.45

    for tamaño in tamaños_template:
        escala = tamaño / 100
        template_escalado = cv.resize(template_inv, None, fx = escala, fy = escala)
        
        detecciones = encontrar_mejor_coincidencia_template(img_gris, template_escalado, threshold)
        num_detecciones = len(detecciones)
        
        if num_detecciones > max_deteccion:
            max_deteccion = num_detecciones
            mejor_medida = template_escalado.shape[::-1]
            mejor_deteccion = detecciones
    
    for pt in mejor_deteccion:
        x, y = pt
        w, h = mejor_medida
        cv.rectangle(img_rgb, (x, y), (x + w, y + h), (0, 255, 0), 2)
    
    cv.imshow("Detecciones", img_rgb)
    cv.waitKey(0)
    cv.destroyAllWindows()


In [31]:

ruta_imagen = 'Material_TP3/images/coca_multi.png'
tamaños_template = [10, 15, 20, 25, 40, 60, 80, 100]
template = cv.imread('Material_TP3/template/pattern.png')
template_gris = cv.cvtColor(template, cv.COLOR_BGR2GRAY)
template_inv = cv.bitwise_not(template_gris)
    
punto_2(ruta_imagen, tamaños_template, template, template_gris, template_inv)

-----------------------------------

# Punto 3

[inicio](#Tabla-de-contenido)

3. (2 puntos) Generalizar el algoritmo del item 2 para todas las imágenes.
Visualizar los resultados con bounding boxes en cada imagen mostrando el nivel de confianza
de la detección.

Este no llegué

-----------------------------------